# Cluster Network Creation

Perform computationally expensive network analysis on a computing cluster. Results are pushed to a data repository on GitHub.

In [1]:
# to allow relative imports
import os
import sys

module = os.path.abspath(os.path.join('..'))
if module not in sys.path:
    sys.path.append(module)

# experiment
from epyc import Lab, JSONLabNotebook, RepeatedExperiment
from data_processing.metrics import MetricExperiment

# generators
from epydemic import PLCNetwork
from model.network.mobility_network import MNGeneratorFromNetworkData as MNG
from model.network.distanced_network import DNGenerator as DNG

# distributions
from model.distributions import discrete_trunc_normal, discrete_trunc_exponential, num_contact_dist

# file utils
from data_processing.data_repo_api import DataRepoAPI
from data_processing.utils import load_network_data_from_files

# other utils
from functools import partial
import numpy as np
from mpmath import polylog

In [2]:
# Some global parameters

N = 1000
CUTOFF = 40
MOBILITY_EXPONENT = 2
n_exp = 10

# Experiment result output dir
output_dir = 'experiment_results'
if not output_dir in os.listdir():
    os.mkdir(output_dir)

# Helper functions
def run_network_lab_experiment(generator, params, file_name, description, n=n_exp,
                               output_dir=output_dir, push_to_repo=True):
    """
    Run a MetricExperiment repeatadly using a network generator.
    """

    file = os.path.join(output_dir, file_name)
    
    nb = JSONLabNotebook(file, create=True, description=description)
    lab = Lab(notebook=nb)
    
    e = MetricExperiment(generator)

    # pass params to the lab
    for k, v in params.items():
        lab[k] = v

    lab.runExperiment(RepeatedExperiment(e, n))
    
    if push_to_repo:
        DataRepoAPI.update_or_create(file_name=file_name, file_path=output_dir)
    
    return lab.results()

def plc_mean(exponent, cutoff):
    """
    Calculate theoretical mean of a power law with cutoff distribution from
    the exponent and the cutoff.
    """
    nom = polylog(exponent-1, np.exp(-1 / cutoff))
    denom = polylog(exponent, np.exp(-1 / cutoff))
    return nom / denom

def estimate_exponent_for_mean(mean, cutoff, tolerance):
    """
    Given the mean and cutoff of a power law with cutoff distribution,
    estimate the corresponding exponent with a given tolerance.
    """
    grid = np.arange(1, 5, 0.01)
    diffs = []
    for i in range(len(grid)):
        diff = abs(mean - plc_mean(grid[i], cutoff))
        if diff < tolerance:
            return grid[i]
        diffs.append(diff)
    
    return grid[diffs.index(min(diffs))]

### Mobility network

In [3]:
# Load the network data

file_names = dict(
    demographics='demographics.pkl',
    comb_pre='comb_counts_pre.pkl',
    comb_post='comb_counts_post.pkl',
    trip_pre='trip_counts_pre.pkl',
    trip_post='trip_counts_post.pkl',
)

network_data = load_network_data_from_files(file_names)

network_data_pre = network_data['pre']
network_data_post = network_data['post']

In [4]:
# Define the generator parameters
params_mobility_pre = dict()
params_mobility_pre[MNG.N] = N
params_mobility_pre[MNG.EXPONENT] = MOBILITY_EXPONENT
params_mobility_pre[MNG.CUTOFF] = CUTOFF
params_mobility_pre[MNG.MULTIPLIER] = False

params_mobility_post = dict()
params_mobility_post[MNG.N] = N
params_mobility_post[MNG.EXPONENT] = MOBILITY_EXPONENT
params_mobility_post[MNG.CUTOFF] = CUTOFF
params_mobility_post[MNG.MULTIPLIER] = True

In [5]:
# Generator instances
mng_pre = MNG(network_data=network_data_pre)
mng_post = MNG(network_data=network_data_post)

In [5]:
# Run experiment
mng_pre_results = run_network_lab_experiment(
    mng_pre, 
    params_mobility_pre, 
    'mobility_pre.json', 
    'Network metrics of Mobility pre'
)


mng_post_results = run_network_lab_experiment(
    mng_post, 
    params_mobility_post, 
    'mobility_post.json', 
    'Network metrics of Mobility post'
)

In [7]:
def extract_mean_deg(results):
    deg_lis = [x['results']['degrees'] for x in results]
    return np.mean([i for s in deg_lis for i in s])

mob_deg_pre = extract_mean_deg(mng_pre_results)
mob_deg_post = extract_mean_deg(mng_post_results)

### PLC network

In [8]:
# Estimate the exponent for the PLC to achieve same degree
#  as the mobility networks

plc_expo_pre = estimate_exponent_for_mean(mob_deg_pre, CUTOFF, 0.01)
plc_expo_post = estimate_exponent_for_mean(mob_deg_post, CUTOFF, 0.01)

In [9]:
# Define the generator parameters
params_plc_pre = dict()
params_plc_pre[PLCNetwork.N] = 10000
params_plc_pre[PLCNetwork.EXPONENT] = plc_expo_pre
params_plc_pre[PLCNetwork.CUTOFF] = CUTOFF

params_plc_post = dict()
params_plc_post[PLCNetwork.N] = 10000
params_plc_post[PLCNetwork.EXPONENT] = plc_expo_post
params_plc_post[PLCNetwork.CUTOFF] = CUTOFF

In [14]:
plc_pre = PLCNetwork()
plc_post = PLCNetwork()

In [10]:
_ = run_network_lab_experiment(plc_pre, params_plc_pre, 'plc_pre.json', 
                               'Network metrics of PLC pre')

_ = run_network_lab_experiment(plc_post, params_plc_post, 'plc_post.json', 
                               'Network metrics of PLC post')

### Distanced network

In [12]:
household_size_dist = partial(discrete_trunc_normal, mu=4.5, std=2)

params_distanced_pre = dict()
params_distanced_pre[DNG.N] = 10000

params_distanced_post = dict()
params_distanced_post[DNG.N] = 10000

In [13]:
distanced_pre = DNG(
    household_size_dist=household_size_dist,
    num_contact_dist=num_contact_dist,
    num_outside_edge_dist=partial(discrete_trunc_exponential, exponent=mob_deg_pre)
)

distanced_post = DNG(
    household_size_dist=household_size_dist,
    num_contact_dist=num_contact_dist,
    num_outside_edge_dist=partial(discrete_trunc_exponential, exponent=mob_deg_post)
)

In [14]:
_ = run_network_lab_experiment(distanced_pre, params_distanced_pre, 'distanced_pre.json', 
                               'Network metrics of PLC pre')

_ = run_network_lab_experiment(distanced_post, params_distanced_post, 'distanced_post.json', 
                               'Network metrics of PLC post')

# Epidemic models

In [6]:
# Import models etc
from epydemic import SEIR
from model.compartmental_models.seir import MonitoredSEIR
from model.compartmental_model.seir import SEIRWithQuarantine as SEIRQ
from model.compartmental_model.seir import MonitoredSEIRWithQuarantine as MonitoredSEIRQ
from model.compartmental_model.seivr import SEIVR, MonitoredSEIVR
from model.compartmental_model.seivr import SEIVRWithQuarantine as SEIVRQ
from model.compartmental_model.seivr import MonitoredSEIVRWithQuarantine as MonitoredSEIVRQ

from epydemic import StochasticDynamics, Monitor

from model.distributions import PowerLawCutoffDist as PLCD

In [16]:
# Helper functions

def calc_p_infect(r0, p_remove, k_mean, k_var):
    """
    Calc p_infect.
    :param r0: basic reproduction number
    :param p_remove: probability of removal
    :param k_mean: Mean of node degree distribution
    :param k_var: Variance of node degree distribution
    :return: p_infect
    """
    return float(p_remove * r0 * (k_mean / (k_var - k_mean)))


def run_stochastic_dynamics_lab(model, generator, params, 
                                file_name, description, n=n_exp,
                                output_dir=output_dir, push_to_repo=True):
    """
    Run a StochasticDynamics experiment repeatadly in a lab.
    """

    file = os.path.join(output_dir, file_name)
    
    nb = JSONLabNotebook(file, create=True, description=description)
    lab = Lab(notebook=nb)
    
    e = StochasticDynamics(model, generator)

    # pass params to the lab
    for k, v in params.items():
        lab[k] = v

    lab.runExperiment(RepeatedExperiment(e, n))
    
    if push_to_repo:
        DataRepoAPI.update_or_create(file_name=file_name, file_path=output_dir)
    
    return lab.results()

## Parameters

We will fix as many COVID-19 specific parameters as we can and leave only policy dependent parameters open for modifications.

We set the following parameters

| Variable | Value | Description | Reference |
| :-- | --: | :-- | :-- |
| R0 | $2.85$ | Basic Reproduction Number | 1) |
| P_INFECT | ... | P. of infection | ... |
| P_REMOVE | $0.1$ | P. of removal | 2) |
| P_SYMPTOPMS | $0.2$ | P. of becoming symptomatic | 3) |
| P_EXPOSED | $0.01$ | P. of initial infection | ... |
| RRR | $0.95$, $0.67$ | Vaccine efficacy | 4), 5), 6) |
| P_VACCINATED | $10^{-3}$ to $10^{-2}$ by $10^{-3}$| P. of getting vaccinated | ... |
| P_VACCINATED_INITIAL | $0$ to $1$ by $0.1$ | P. of initial vaccination | ... |
| P_QUARANTINE | $0$ to $1$ by $0.1$ | P. of rewiring | ... |


1) https://journals.plos.org/plosone/article?id=10.1371/journal.pone.0249271

2) https://www.thelancet.com/journals/lanmic/article/PIIS2666-5247(20)30172-5/fulltext

3) https://doi.org/10.3389/fpubh.2020.598547

4) https://www.nejm.org/doi/full/10.1056/NEJMoa2034577

5) https://www.nejm.org/doi/full/10.1056/nejmoa2035389

6) https://www.sciencedirect.com/science/article/pii/S0140673620326611

In [5]:
# Fixed
R0 = 2.85
_P_INFECT = None  # to calculate later 
P_REMOVE = 0.1
P_SYMPTOMS = 0.2
P_EXPOSED = 0.01

_RRR = [0.95, 0.67]

# Free
_P_VACCINATED = np.arange(0.001, 0.011, 0.001)
_P_VACCINATED_INITAL = np.arange(0, 1.1, 0.1)
_P_QUARANTINE = np.arange(0, 1.1, 0.1)

In [18]:
# Fix parameters as far as possible

params_seir = dict()
params_seir[SEIR.P_SYMPTOMS] = P_SYMPTOMS
params_seir[SEIR.P_REMOVE] = P_REMOVE
params_seir[SEIR.P_EXPOSED] = P_EXPOSED

params_seivr = dict()
params_seivr[SEIVR.P_SYMPTOMS] = P_SYMPTOMS
params_seivr[SEIVR.P_REMOVE] = P_REMOVE
params_seivr[SEIVR.P_EXPOSED] = P_EXPOSED

params_monitor = dict()
params_monitor[Monitor.DELTA] = 10

In [ ]:
# Calculate P_INFECT

# Mobility
plcd = PLCD(MOBILITY_EXPONENT, CUTOFF)
P_INFECT_MOBILITY = calc_p_infect(R0, P_REMOVE, plcd.mean, plcd.var)

# PLC
plcd = PLCD(plc_expo_pre, CUTOFF)
P_INFECT_PLC = calc_p_infect(R0, P_REMOVE, plcd.mean, plcd.var)

# Distanced
mean = mob_deg_pre
var = mean ** 2
P_INFECT_DISTANCED = calc_p_infect(R0, P_REMOVE, mean, var)

# clean up
del mean, var, plcd

### SEIR & SEIR_Q

#### SEIR on Mobility

In [20]:
# params
params_pre = params_mobility_pre | params_seir | params_monitor
params_pre[SEIR.P_INFECT_SYMPTOMATIC] = P_INFECT_MOBILITY
params_pre[SEIR.P_INFECT_ASYMPTOMATIC] = P_INFECT_MOBILITY / 2

params_post = params_mobility_post | params_seir | params_monitor
params_post[SEIR.P_INFECT_SYMPTOMATIC] = P_INFECT_MOBILITY
params_post[SEIR.P_INFECT_ASYMPTOMATIC] = P_INFECT_MOBILITY / 2

In [22]:
_ = run_stochastic_dynamics_lab(
    MonitoredSEIR(),
    mng_pre, 
    params_pre, 
    'seir_mobility_pre.json',
    'SEIR simulation with Mobility Network (Pre)',
)

_ = run_stochastic_dynamics_lab(
    MonitoredSEIR(), 
    mng_post, 
    params_post, 
    'seir_mobility_post.json',
    'SEIR simulation with Mobility Network (Post)'
)

In [ ]:
# reset params
del params_pre, params_post

#### SEIR_Q on Mobility

In [24]:
params_pre = params_mobility_pre | params_seir | params_monitor
params_pre[SEIRQ.P_INFECT_SYMPTOMATIC] = P_INFECT_MOBILITY
params_pre[SEIRQ.P_INFECT_ASYMPTOMATIC] = P_INFECT_MOBILITY / 2
params_pre[SEIRQ.P_QUARANTINE] = _P_QUARANTINE

params_post = params_mobility_post | params_seir | params_monitor
params_post[SEIRQ.P_INFECT_SYMPTOMATIC] = P_INFECT_MOBILITY
params_post[SEIRQ.P_INFECT_ASYMPTOMATIC] = P_INFECT_MOBILITY / 2
params_post[SEIRQ.P_QUARANTINE] = _P_QUARANTINE

In [ ]:
_ = run_stochastic_dynamics_lab(
    MonitoredSEIRQ(), 
    mng_pre, 
    params_pre, 
    'seirq_mobility_pre.json',
    'SEIR_Q simulation with Mobility Network (Pre)',
)

_ = run_stochastic_dynamics_lab(
    MonitoredSEIRQ(), 
    mng_post, 
    params_post, 
    'seirq_mobility_post.json',
    'SEIR_Q simulation with Mobility Network (Post)'
)

In [ ]:
# reset params
del params_pre, params_post

#### SEIR on PLC

In [ ]:
params_pre = params_plc_pre | params_seir | params_monitor
params_pre[SEIR.P_INFECT_SYMPTOMATIC] = P_INFECT_PLC
params_pre[SEIR.P_INFECT_ASYMPTOMATIC] = P_INFECT_PLC / 2

params_post = params_plc_post | params_seir | params_monitor
params_post[SEIR.P_INFECT_SYMPTOMATIC] = P_INFECT_PLC
params_post[SEIR.P_INFECT_ASYMPTOMATIC] = P_INFECT_PLC / 2

In [ ]:
_ = run_stochastic_dynamics_lab(
    MonitoredSEIR(), 
    PLCNetwork(), 
    params_pre, 
    'seir_plc_pre.json',
    'SEIR simulation with PLC Network (Pre)',
)

_ = run_stochastic_dynamics_lab(
    MonitoredSEIR(), 
    PLCNetwork(), 
    params_post, 
    'seir_plc_post.json',
    'SEIR simulation with PLC Network (Post)'
)

In [ ]:
del params_pre, params_post

#### SEIR_Q on PLC

In [ ]:
params_pre = params_plc_pre | params_seir | params_monitor
params_pre[SEIRQ.P_INFECT_SYMPTOMATIC] = P_INFECT_PLC
params_pre[SEIRQ.P_INFECT_ASYMPTOMATIC] = P_INFECT_PLC / 2
params_pre[SEIRQ.P_QUARANTINE] = _P_QUARANTINE

params_post = params_plc_post | params_seir | params_monitor
params_post[SEIRQ.P_INFECT_SYMPTOMATIC] = P_INFECT_PLC
params_post[SEIRQ.P_INFECT_ASYMPTOMATIC] = P_INFECT_PLC / 2
params_post[SEIRQ.P_QUARANTINE] = _P_QUARANTINE

In [ ]:
_ = run_stochastic_dynamics_lab(
    MonitoredSEIRQ(), 
    PLCNetwork(), 
    params_pre, 
    'seirq_plc_pre.json',
    'SEIR_Q simulation with PLC Network (Pre)',
)

_ = run_stochastic_dynamics_lab(
    MonitoredSEIRQ(), 
    PLCNetwork(), 
    params_post, 
    'seirq_plc_post.json',
    'SEIR_Q simulation with PLC Network (Post)'
)

In [ ]:
del params_pre, params_post

#### SEIR on Distanced

In [ ]:
params_pre = params_distanced_pre | params_seir | params_monitor
params_pre[SEIR.P_INFECT_SYMPTOMATIC] = P_INFECT_DISTANCED
params_pre[SEIR.P_INFECT_ASYMPTOMATIC] = P_INFECT_DISTANCED / 2

params_post = params_distanced_post | params_seir | params_monitor
params_post[SEIR.P_INFECT_SYMPTOMATIC] = P_INFECT_DISTANCED
params_post[SEIR.P_INFECT_ASYMPTOMATIC] = P_INFECT_DISTANCED / 2

In [ ]:
_ = run_stochastic_dynamics_lab(
    MonitoredSEIR(), 
    distanced_pre, 
    params_pre, 
    'seir_distanced_pre.json',
    'SEIR simulation with Distanced Network (Pre)',
)

_ = run_stochastic_dynamics_lab(
    MonitoredSEIR(), 
    distanced_post, 
    params_post, 
    'seir_distanced_post.json',
    'SEIR simulation with Distanced Network (Post)'
)

In [ ]:
del params_pre, params_post

#### SEIR_Q on Distanced

In [ ]:
params_pre = params_distanced_pre | params_seir | params_monitor
params_pre[SEIRQ.P_INFECT_SYMPTOMATIC] = P_INFECT_DISTANCED
params_pre[SEIRQ.P_INFECT_ASYMPTOMATIC] = P_INFECT_DISTANCED / 2
params_pre[SEIRQ.P_QUARANTINE] = _P_QUARANTINE

params_post = params_distanced_post | params_seir | params_monitor
params_post[SEIRQ.P_INFECT_SYMPTOMATIC] = P_INFECT_DISTANCED
params_post[SEIRQ.P_INFECT_ASYMPTOMATIC] = P_INFECT_DISTANCED / 2
params_post[SEIRQ.P_QUARANTINE] = _P_QUARANTINE

In [ ]:
_ = run_stochastic_dynamics_lab(
    MonitoredSEIRQ(), 
    distanced_pre, 
    params_pre, 
    'seirq_distanced_pre.json',
    'SEIR_Q simulation with Distanced Network (Pre)',
)

_ = run_stochastic_dynamics_lab(
    MonitoredSEIRQ(), 
    distanced_post, 
    params_post, 
    'seirq_distanced_post.json',
    'SEIR_Q simulation with Distanced Network (Post)'
)

In [ ]:
del params_pre, params_post

### SEIVR & SEIVR_Q

#### SEIVR on Mobility

In [7]:
params_pre = params_mobility_pre | params_seivr | params_monitor
params_pre[SEIVR.P_INFECT_SYMPTOMATIC] = P_INFECT_MOBILITY
params_pre[SEIVR.P_INFECT_ASYMPTOMATIC] = P_INFECT_MOBILITY / 2
params_pre[SEIVR.P_VACCINATED] = _P_VACCINATED
params_pre[SEIVR.P_VACCINATED_INITIAL] = _P_VACCINATED_INITAL
params_pre[SEIVR.VACCINE_RRR] = _RRR

params_post = params_mobility_post | params_seivr | params_monitor
params_post[SEIVR.P_INFECT_SYMPTOMATIC] = P_INFECT_MOBILITY
params_post[SEIVR.P_INFECT_ASYMPTOMATIC] = P_INFECT_MOBILITY / 2
params_post[SEIVR.P_VACCINATED] = _P_VACCINATED
params_post[SEIVR.P_VACCINATED_INITIAL] = _P_VACCINATED_INITAL
params_post[SEIVR.VACCINE_RRR] = _RRR

NameError: name 'params_seivr' is not defined

In [ ]:
_ = run_stochastic_dynamics_lab(
    MonitoredSEIVR(), 
    mng_pre, 
    params_pre, 
    'seivr_mobility_pre.json',
    'SEIVR simulation with Mobility Network (Pre)',
)

_ = run_stochastic_dynamics_lab(
    MonitoredSEIVR(), 
    mng_post, 
    params_post, 
    'seivr_mobility_post.json',
    'SEIVR simulation with Mobility Network (Post)'
)

In [ ]:
del params_pre, params_post

#### SEIVR_Q on Mobility

In [6]:
params_pre = params_mobility_pre | params_seivr | params_monitor
params_pre[SEIVRQ.P_INFECT_SYMPTOMATIC] = P_INFECT_MOBILITY
params_pre[SEIVRQ.P_INFECT_ASYMPTOMATIC] = P_INFECT_MOBILITY / 2
params_pre[SEIVRQ.P_VACCINATED] = _P_VACCINATED
params_pre[SEIVRQ.P_VACCINATED_INITIAL] = _P_VACCINATED_INITAL
params_pre[SEIVRQ.VACCINE_RRR] = _RRR
params_pre[SEIVRQ.P_QUARANTINE] = _P_QUARANTINE

params_post = params_mobility_post | params_seivr | params_monitor
params_post[SEIVRQ.P_INFECT_SYMPTOMATIC] = P_INFECT_MOBILITY
params_post[SEIVRQ.P_INFECT_ASYMPTOMATIC] = P_INFECT_MOBILITY / 2
params_post[SEIVRQ.P_VACCINATED] = _P_VACCINATED
params_post[SEIVRQ.P_VACCINATED_INITIAL] = _P_VACCINATED_INITAL
params_post[SEIVRQ.VACCINE_RRR] = _RRR
params_post[SEIVRQ.P_QUARANTINE] = _P_QUARANTINE

NameError: name 'params_seivr' is not defined

In [ ]:
_ = run_stochastic_dynamics_lab(
    MonitoredSEIVRQ(), 
    mng_pre, 
    params_pre, 
    'seivrq_mobility_pre.json',
    'SEIVR_Q simulation with Mobility Network (Pre)',
)

_ = run_stochastic_dynamics_lab(
    MonitoredSEIVRQ(), 
    mng_post, 
    params_post, 
    'seivrq_mobility_post.json',
    'SEIVR_Q simulation with Mobility Network (Post)'
)

In [ ]:
del params_pre, params_post

#### SEIVR on PLC

In [ ]:
params_pre = params_plc_pre | params_seivr | params_monitor
params_pre[SEIVR.P_INFECT_SYMPTOMATIC] = P_INFECT_PLC
params_pre[SEIVR.P_INFECT_ASYMPTOMATIC] = P_INFECT_PLC / 2
params_pre[SEIVR.P_VACCINATED] = _P_VACCINATED
params_pre[SEIVR.P_VACCINATED_INITIAL] = _P_VACCINATED_INITAL
params_pre[SEIVR.VACCINE_RRR] = _RRR

params_post = params_plc_post | params_seivr | params_monitor
params_post[SEIVR.P_INFECT_SYMPTOMATIC] = P_INFECT_PLC
params_post[SEIVR.P_INFECT_ASYMPTOMATIC] = P_INFECT_PLC / 2
params_post[SEIVR.P_VACCINATED] = _P_VACCINATED
params_post[SEIVR.P_VACCINATED_INITIAL] = _P_VACCINATED_INITAL
params_post[SEIVR.VACCINE_RRR] = _RRR

In [ ]:
_ = run_stochastic_dynamics_lab(
    MonitoredSEIVR(), 
    PLCNetwork(), 
    params_pre, 
    'seivr_plc_pre.json',
    'SEIVR simulation with PLC Network (Pre)',
)

_ = run_stochastic_dynamics_lab(
    MonitoredSEIVR(), 
    PLCNetwork(), 
    params_post, 
    'seivr_plc_post.json',
    'SEIVR simulation with PLC Network (Post)'
)

In [ ]:
del params_pre, params_post

#### SEIVR_Q on PLC

In [ ]:
params_pre = params_plc_pre | params_seivr | params_monitor
params_pre[SEIVRQ.P_INFECT_SYMPTOMATIC] = P_INFECT_PLC
params_pre[SEIVRQ.P_INFECT_ASYMPTOMATIC] = P_INFECT_PLC / 2
params_pre[SEIVRQ.P_VACCINATED] = _P_VACCINATED
params_pre[SEIVRQ.P_VACCINATED_INITIAL] = _P_VACCINATED_INITAL
params_pre[SEIVRQ.VACCINE_RRR] = _RRR
params_pre[SEIVRQ.P_QUARANTINE] = _P_QUARANTINE

params_post = params_plc_post | params_seivr | params_monitor
params_post[SEIVRQ.P_INFECT_SYMPTOMATIC] = P_INFECT_PLC
params_post[SEIVRQ.P_INFECT_ASYMPTOMATIC] = P_INFECT_PLC / 2
params_post[SEIVRQ.P_VACCINATED] = _P_VACCINATED
params_post[SEIVRQ.P_VACCINATED_INITIAL] = _P_VACCINATED_INITAL
params_post[SEIVRQ.VACCINE_RRR] = _RRR
params_post[SEIVRQ.P_QUARANTINE] = _P_QUARANTINE

In [ ]:
_ = run_stochastic_dynamics_lab(
    MonitoredSEIVRQ(), 
    PLCNetwork(), 
    params_pre, 
    'seivrq_plc_pre.json',
    'SEIVR_Q simulation with PLC Network (Pre)',
)

_ = run_stochastic_dynamics_lab(
    MonitoredSEIVRQ(), 
    PLCNetwork(), 
    params_post, 
    'seivrq_plc_post.json',
    'SEIVR_Q simulation with PLC Network (Post)'
)

In [ ]:
del params_pre, params_post

#### SEIVR on Distanced

In [ ]:
params_pre = params_distanced_pre | params_seivr | params_monitor
params_pre[SEIVR.P_INFECT_SYMPTOMATIC] = P_INFECT_DISTANCED
params_pre[SEIVR.P_INFECT_ASYMPTOMATIC] = P_INFECT_DISTANCED / 2
params_pre[SEIVR.P_VACCINATED] = _P_VACCINATED
params_pre[SEIVR.P_VACCINATED_INITIAL] = _P_VACCINATED_INITAL
params_pre[SEIVR.VACCINE_RRR] = _RRR

params_post = params_distanced_post | params_seivr | params_monitor
params_post[SEIVR.P_INFECT_SYMPTOMATIC] = P_INFECT_DISTANCED
params_post[SEIVR.P_INFECT_ASYMPTOMATIC] = P_INFECT_DISTANCED / 2
params_post[SEIVR.P_VACCINATED] = _P_VACCINATED
params_post[SEIVR.P_VACCINATED_INITIAL] = _P_VACCINATED_INITAL
params_post[SEIVR.VACCINE_RRR] = _RRR

In [ ]:
_ = run_stochastic_dynamics_lab(
    MonitoredSEIVR(), 
    distanced_pre, 
    params_pre, 
    'seivr_distanced_pre.json',
    'SEIVR simulation with Distanced Network (Pre)',
)

_ = run_stochastic_dynamics_lab(
    MonitoredSEIVR(), 
    distanced_post, 
    params_post, 
    'seivr_distanced_post.json',
    'SEIVR simulation with Distanced Network (Post)'
)

In [ ]:
del params_pre, params_post

#### SEIVR_Q on Distanced

In [ ]:
params_pre = params_distanced_pre | params_seivr | params_monitor
params_pre[SEIVRQ.P_INFECT_SYMPTOMATIC] = P_INFECT_DISTANCED
params_pre[SEIVRQ.P_INFECT_ASYMPTOMATIC] = P_INFECT_DISTANCED / 2
params_pre[SEIVRQ.P_VACCINATED] = _P_VACCINATED
params_pre[SEIVRQ.P_VACCINATED_INITIAL] = _P_VACCINATED_INITAL
params_pre[SEIVRQ.VACCINE_RRR] = _RRR
params_pre[SEIVRQ.P_QUARANTINE] = _P_QUARANTINE

params_post = params_distanced_post | params_seivr | params_monitor
params_post[SEIVRQ.P_INFECT_SYMPTOMATIC] = P_INFECT_DISTANCED
params_post[SEIVRQ.P_INFECT_ASYMPTOMATIC] = P_INFECT_DISTANCED / 2
params_post[SEIVRQ.P_VACCINATED] = _P_VACCINATED
params_post[SEIVRQ.P_VACCINATED_INITIAL] = _P_VACCINATED_INITAL
params_post[SEIVRQ.VACCINE_RRR] = _RRR
params_post[SEIVRQ.P_QUARANTINE] = _P_QUARANTINE

In [ ]:
_ = run_stochastic_dynamics_lab(
    MonitoredSEIVRQ(), 
    distanced_pre, 
    params_pre, 
    'seivrq_distanced_pre.json',
    'SEIVR_Q simulation with Distanced Network (Pre)',
)

_ = run_stochastic_dynamics_lab(
    MonitoredSEIVRQ(), 
    distanced_post, 
    params_post, 
    'seivrq_distanced_post.json',
    'SEIVR_Q simulation with Distanced Network (Post)'
)

In [ ]:
del params_pre, params_post